In [1]:
import pandas as pd
import numpy as np
import os
import sklearn.metrics
import pickle as pickle



In [2]:
num_to_label = {0: 'no_relation', 1: 'org:top_members/employees', 2: 'org:members', 3: 'org:product', 4: 'per:title', 5: 'org:alternate_names', 6: 'per:employee_of', 7: 'org:place_of_headquarters', 8: 'per:product', 9: 'org:number_of_employees/members', 10: 'per:children', 11: 'per:place_of_residence', 12: 'per:alternate_names', 13: 'per:other_family', 14: 'per:colleagues', 15: 'per:origin', 16: 'per:siblings', 17: 'per:spouse', 18: 'org:founded', 19: 'org:political/religious_affiliation', 20: 'org:member_of', 21: 'per:parents', 22: 'org:dissolved', 23: 'per:schools_attended', 24: 'per:date_of_death', 25: 'per:date_of_birth', 26: 'per:place_of_birth', 27: 'per:place_of_death', 28: 'org:founded_by', 29: 'per:religion'}

In [44]:
def klue_re_micro_f1(preds, labels):
    """KLUE-RE micro f1 (except no_relation)"""
    label_list = ['no_relation', 'org:top_members/employees', 'org:members',
       'org:product', 'per:title', 'org:alternate_names',
       'per:employee_of', 'org:place_of_headquarters', 'per:product',
       'org:number_of_employees/members', 'per:children',
       'per:place_of_residence', 'per:alternate_names',
       'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
       'per:spouse', 'org:founded', 'org:political/religious_affiliation',
       'org:member_of', 'per:parents', 'org:dissolved',
       'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
       'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
       'per:religion']
    no_relation_label_idx = label_list.index("no_relation")
    preds = label_to_num(preds.values)
    labels = label_to_num(labels.values)

    label_indices = list(range(len(label_list)))
    label_indices.remove(no_relation_label_idx)
    return sklearn.metrics.f1_score(labels, preds, average="micro", labels=label_indices) * 100.0


def label_to_num2(label):
  num_label = []
  with open('../code/dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
  # for v in label:
  #   num_label.append(dict_label_to_num[v])
  
  return dict_label_to_num[label]
def label_to_num(label):
  num_label = []
  with open('../code/dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
  for v in label:
    num_label.append(dict_label_to_num[v])
  
  return num_label

In [22]:
def mixmaster(df1, df2, weight1, weight2):
    result = []
    df1= np.array(list(map(float, df1[1:-1].split(', ')))) * weight1
    df2= np.array(list(map(float, df2[1:-1].split(', ')))) * weight2
    result =np.mean([df1, df2], axis=0)
    result = result/result.sum()
    result = '[' + ', '.join(map(str, result)) + ']'
    return result

In [23]:
def change_label(x):
    x= np.array(list(map(float, x[1:-1].split(', '))))
    return num_to_label[np.argmax(x)]

In [45]:
def ensemble(folder_name):
    file_list = os.listdir(folder_name)
    result = pd.read_csv(f'{folder_name}/{file_list[0]}')
    li=[[] for i in range(7765)]
    li2=[]
    for file_name in file_list:
        ens1 = pd.read_csv(f'{folder_name}/{file_name}')
        print(ens1.columns)
        for i,j in enumerate(ens1.pred_label.values):
            li[i].append(label_to_num2(j))
    print(li)
    li2=[num_to_label[max(x,key= x.count)] for x in li]
    print(li2)

    result.pred_label = li2

    print(result.pred_label)
    return result
    
    

In [46]:
correct = pd.read_csv('../data/dataset/test/dev_final.csv')
folder_name = './ensemble_hard'

ens_result = ensemble(folder_name)
ens_result= pd.concat([ens_result,correct],axis=1)
print(ens_result.label)
klue_re_micro_f1(ens_result.pred_label,ens_result.label)

Index(['id', 'pred_label', 'probs'], dtype='object')
Index(['id', 'pred_label', 'probs'], dtype='object')
Index(['id', 'pred_label', 'probs'], dtype='object')
Index(['id', 'pred_label', 'probs'], dtype='object')
[[3, 3, 3, 3], [12, 12, 12, 12], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [20, 0, 0, 20], [6, 6, 6, 6], [17, 17, 17, 17], [2, 2, 2, 2], [1, 1, 1, 1], [0, 0, 0, 0], [22, 22, 22, 22], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [14, 14, 14, 0], [12, 12, 12, 12], [24, 0, 0, 0], [11, 0, 0, 0], [3, 3, 3, 3], [0, 14, 17, 0], [1, 1, 1, 1], [1, 1, 1, 1], [6, 6, 6, 6], [0, 0, 0, 0], [0, 0, 0, 7], [0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [7, 7, 7, 7], [11, 11, 15, 11], [0, 0, 0, 0], [0, 0, 0, 0], [0, 4, 4, 0], [0, 0, 0, 0], [3, 3, 3, 3], [0, 0, 0, 0], [1, 1, 1, 1], [0, 0, 0, 0], [0, 0, 0, 0], [6, 6, 6, 6], [20, 20, 20, 20], [0, 0, 0, 0], [4, 4, 4, 4], [1, 1, 1, 1], [6, 6, 6, 4], [0, 0, 0, 0], [0, 0, 2, 0], [3, 3, 3, 3], [0, 0, 0, 0], [0, 0, 0, 0], [14, 14, 14,

75.4836663495084